In [4]:
import torch
from torch import nn
import math

In [ ]:
x=torch.rand(128,32,512)
print(x)

tensor([[[-0.6386, -0.9859, -1.7846, -0.9976],
         [-0.8222,  0.7146,  0.8686, -0.0655],
         [ 0.2177, -1.8170,  1.3107,  0.0509]],

        [[ 2.5254, -0.2969, -0.1884, -0.6320],
         [ 1.6079, -0.2558, -0.0389,  1.1212],
         [-1.2021,  0.7071, -1.4883,  0.5893]]])
原始形状: torch.Size([2, 3, 4])
tensor([[[-0.6386, -0.8222,  0.2177],
         [-0.9859,  0.7146, -1.8170],
         [-1.7846,  0.8686,  1.3107],
         [-0.9976, -0.0655,  0.0509]],

        [[ 2.5254,  1.6079, -1.2021],
         [-0.2969, -0.2558,  0.7071],
         [-0.1884, -0.0389, -1.4883],
         [-0.6320,  1.1212,  0.5893]]])
permute(0, 2, 1)后形状: torch.Size([2, 4, 3])
tensor([[[-0.6386,  2.5254],
         [-0.8222,  1.6079],
         [ 0.2177, -1.2021]],

        [[-0.9859, -0.2969],
         [ 0.7146, -0.2558],
         [-1.8170,  0.7071]],

        [[-1.7846, -0.1884],
         [ 0.8686, -0.0389],
         [ 1.3107, -1.4883]],

        [[-0.9976, -0.6320],
         [-0.0655,  1.1212],
         [

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, n_head):
        super().__init__()
        self.d_model = d_model
        self.n_head = n_head
        self.w_q = nn.Linear(d_model, d_model)
        self.w_k = nn.Linear(d_model, d_model)
        self.w_v = nn.Linear(d_model, d_model)
        self.w_combine = nn.Linear(d_model, d_model)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, q, k, v, mask=None):
        batch, time, dimension = q.shape
        head_d = self.d_model // self.n_head
        q, k, v = self.w_q(q), self.w_k(k), self.w_v(v)
        q = q.view(batch, time, self.n_head, head_d).permute(0, 2, 1, 3)
        k = k.view(batch, time, self.n_head, head_d).permute(0, 2, 1, 3)
        v = v.view(batch, time, self.n_head, head_d).permute(0, 2, 1, 3)
        score = q @ k.transpose(2, 3) / math.sqrt(head_d)
        if mask is not None:
            score = score.masked_fill(mask == 0, -10000)
        score = self.softmax(score) @ v
        score = score.permute(0, 2, 1, 3).contiguous().view(batch, time, dimension)
        out = self.w_combine(score)
        return out
